In [1]:
from wann_genetic.tools import ExperimentSeries, mean_comparison
from wann_genetic.individual.util import num_used_activation_functions
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from itertools import permutations
import pandas as pd
from wann_genetic.individual.numpy import Individual
from scipy.stats import ttest_rel, ttest_ind
from tabulate import tabulate
from itertools import product

In [2]:
# set output format to svg
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

In [3]:
series = ExperimentSeries.from_spec_file("../data/iris_act_funcs/all")
series.discover_data_dir()
print (f"{len(series)} experiments in series.")
df = series.assemble_stats()
function_names = [name[:4] for name, _ in Individual.Phenotype.available_act_functions]
long_function_names = {name[:4]: name for name, _ in Individual.Phenotype.available_act_functions}

1576 experiments in series.


In [4]:
prefix = 'n_nodes_with_act_func_'

for distribution, signed in product(['uniform', 'lognormal'], [True, False]):
    signed_string = "enabled" if signed else "disabled"

    accuracies = dict()
    
    selection = (df['distribution/distribution'] == distribution) & (df['signed']==signed)
    

    relative_numbers_all = dict()
    relative_numbers_hidden = dict()

    for c in series.configurations():
        flat_values = series.flat_values(c)
        if flat_values['distribution/distribution'] != distribution or flat_values['signed'] != signed:
            continue
    
        n_act_funcs = len(flat_values['activation_functions'])
        env = series.configuration_env(c)
        with env.open_data():
            env.load_hof()
            for ind in env.hall_of_fame:
                ind.express()
                n_hidden = ind.network.n_hidden
                n_all = ind.network.n_hidden + ind.network.n_out
                
                for k,v in num_used_activation_functions(ind.genes.nodes, ind.network.enabled_act_functions).items():
                    k = k[len(prefix):]

                    if k not in relative_numbers_all:
                        relative_numbers_all[k] = list()

                    relative_numbers_all[k].append(v / n_all - 1/n_act_funcs)
                    
                for k,v in num_used_activation_functions(ind.genes.nodes[ind.genes.nodes['out']==False], ind.network.enabled_act_functions).items():
                    k = k[len(prefix):]

                    if k not in relative_numbers_hidden:
                        relative_numbers_hidden[k] = list()

                    if n_hidden != 0:
                        relative_numbers_hidden[k].append(v / n_hidden - 1/n_act_funcs)

    for index, row in df[selection].iterrows():
        accuracies[tuple(sorted(row['activation_functions']))] = row['MAX:accuracy.mean']

    paired_samples = list()

    for (co1, acc1), (co2, acc2) in permutations(accuracies.items(), 2):
        if (len(co1) == len(co2) - 1) and set(co1).issubset(co2):
            element, = [e for e in co2 if e not in co1]
            paired_samples.append({'activation function': function_names[element], 'acc with': acc2, 'acc without': acc1})

    paired_samples = pd.DataFrame(paired_samples)

    rows = list()
    for fn in function_names:
        data = paired_samples[paired_samples['activation function'] == fn]
        t, p = ttest_rel(data['acc with'], data['acc without'])
        
        rel_occ_all = np.array(relative_numbers_all[long_function_names[fn]]).mean()
        rel_occ_hidden = np.array(relative_numbers_hidden[long_function_names[fn]]).mean()
        
        cells = [long_function_names[fn], f"{t:.2}", f"{p:.2%}", f"{rel_occ_all:.2%}", f"{rel_occ_hidden:.2%}"]
        if p < 0.05:
            cells = [f"**{cell}**" for cell in cells]
        rows.append(cells)
    print(tabulate(rows, tablefmt='grid', headers=('activation function', 't-value', 'p-value', 'rel. occurrence', 'rel. occurrence\n(hidden only)')))
    
    signed = str(signed).lower()
    print (f"\nTable: {distribution.capitalize()} distribution with edge signs {signed_string}. \\label{{tbl:act_funcs_{distribution}_signs_{signed}}}\n\n")

+-----------------------+-----------+-----------+-------------------+-------------------+
| activation function   | t-value   | p-value   | rel. occurrence   | rel. occurrence   |
|                       |           |           |                   | (hidden only)     |
+=======================+===========+===========+===================+===================+
| **relu**              | **-2.1**  | **4.06%** | **-7.44%**        | **-6.12%**        |
+-----------------------+-----------+-----------+-------------------+-------------------+
| sigmoid               | 1.2       | 24.06%    | -7.07%            | -3.43%            |
+-----------------------+-----------+-----------+-------------------+-------------------+
| **tanh**              | **2.2**   | **3.12%** | **-1.08%**        | **7.14%**         |
+-----------------------+-----------+-----------+-------------------+-------------------+
| gaussian (standard)   | -0.11     | 90.91%    | 14.95%            | 5.02%             |
+---------

In [5]:
print(series.hyperparam_table(tablefmt='grid'))

+----------------------------------------------------+--------------------------------------+
| Parameter                                          | Value                                |
+====================================================+======================================+
| Population size                                    | 100                                  |
+----------------------------------------------------+--------------------------------------+
| Number of generations                              | 200                                  |
+----------------------------------------------------+--------------------------------------+
| Initial population                                 | fully connected                      |
|                                                    | (edges have 5% chance of             |
|                                                    | being enabled)                       |
+----------------------------------------------------+------